# Lagrangian Relaxation Implementation
### Supply chain project
#### Team Members: Ghizlaine Bennani, Chuiyi (Tracy) Liu, Tate Campbell, Abhishek Singh

In [33]:
#Loading the libraries for computation
import pandas as pd, numpy as np

In [34]:
#Loading the distance datasets & generating matrix from them
distance_49 = np.array(pd.read_csv('49 NodeDistanceData.txt')) # 49 Distance Node
distance_49 = distance_49.reshape(49,49)
distance_88 = np.array(pd.read_csv('88 NodeDistanceData.txt')) # 88 Distance Node
distance_88 = distance_88.reshape(88,88)
distance_150 = np.array(pd.read_csv('150 NodeDistanceData.txt')) # 150 Distance Node
distance_150 = distance_150.reshape(150,150)

#Loading the demand datasets & selecting only the first column for our problem
demand_49 = pd.read_csv('49 NodeDemandData.txt', sep='\t',  header = True) # 49 Demand Node
demand_49 = demand_49.icol(0) 
demand_88 = pd.read_csv('88 NodeDemandData.txt', sep='\t',  header = True) # 88 Demand Node
demand_88 = demand_88.icol(0) 
demand_150 = pd.read_csv('150 NodeDemandData.txt', sep='\t',  header = True)# 150 Demand Node
demand_150 = demand_150.icol(0)           


In [35]:
def pre_process(dis_matrix, Dc = 700):
    """
    This function will preprocess the distance. 
    :param dis_matrix: a matrix with rows as node and columns as node and the entry is the distance
    :param Dc: the availabel distance that can be achieved 
    :return pro_matrix: a matrix the entry where larger than the Dc will be assigned 0, otherwise 1.
    """
    m, n = dis_matrix.shape
    pro_matrix = np.zeros((m, n))
    pro_matrix[dis_matrix < Dc] = 1
    return pro_matrix

In [36]:
#Preprocessing all the distance matrices
distance_49 = pre_process(distance_49)
distance_88 = pre_process(distance_88)
distance_150 = pre_process(distance_150)

In [37]:
def lagrangian_relaxation(demand_data, distance_data, lambda_i):
    """
    For performing LR Iterations with the given inputs, to update Alpha & t
    ouput: Upper Bound & Lower Bound
    """
    h_i, Z_i = demand_data, np.zeros(len(h_i)) ## subproblem 1
    Z_i[h_i - lambda_i > 0] = 1 #Tagging as 1 for the breakoff points
    sub1_value = np.dot((h_i - lambda_i), Z_i)
    
    a_ij_lambda_i = np.dot(distance_data, lambda_i) ## subproblem 2
    X_i = np.zeros(len(a_ij_lambda_i)) # X_i is the solution to subproblem 2

    X_i[X_i == max(X_i)] = 1
    sub2_value = np.dot(a_ij_lambda_i, X_i)
    
    upper_bound = sub1_value + sub2_value
    lower_bound = sum(h_i[np.dot(a_ij_lambda_i, X_i) - Z_i > 0])
    
    return X_i, Z_i, upper_bound, lower_bound 

In [38]:
def update_lambda(alpha, upper_bound, lower_bound, a_ij, X_i, Z_i, lambda_i):
    """
    This function returns the value of updated lambda using previous values
    """
    t = alpha * (upper_bound - lower_bound)/ sum((np.dot(a_ij, X_i) - Z_i)^2)
    lambda_i = max(0, lambda_i - t*(np.dot(a_ij, X_i) - Z_i))
    return lambda_i

In [39]:
def iterator(demand, distance, supply_points = 10):
    """
    Getting started for all the different nodes
    :param demand: demand array
    :param distance: distance matrix
    """
    n, alpha = 1, 2.0
    lambda_i = np.mean(demand) + 0.5 * (demand - np.mean(demand))
    
    max_iterations, min_alpha = 500, 0.01 #To ensure convergence

    X_i, Z_i, upper_bound, lower_bound = lagrangian_relaxation(demand, distance, lambda_i, supply_points)
    best_lower_bound, best_upper_bound = lower_bound, upper_bound #Updating UB & LB after each iteration

    improvement_iteration = 0
    
    converged = False
    
    while not converged:    
        X_i, Z_i, upper_bound, lower_bound = lagrangian_relaxation(demand, distance, lambda_i)

        if upper_bound >= best_upper_bound: 
            improvement_iteration += 1

        best_upper_bound = min(best_upper_bound, upper_bound)
        best_lower_bound = max(best_lower_bound, lower_bound)

        if best_upper_bound == best_lower_bound or n > max_iterations or alpha < min_alpha:
            print best_upper_bound == best_lower_bound, n > max_iterations, alpha < min_alpha
            return X_i, Z_i, upper_bound, lower_bound  

        elif improvement_iteration == 4:
            alpha = alpha/2.0

        else:
            lambda_i = update_lambda(alpha, upper_bound, lower_bound, distance, X_i, Z_i, lambda_i)

        n += 1 
        print n
        if improvement_iteration >= max_iterations  or  improvement_iteration <= min_alpha:
            converged = True 
        
        demand_met = np.dot(X_i, demand)
        
    return demand_met

In [40]:
# For the 49 Node cluster
iterator(demand = demand_49, distance = distance_49, supply_points = 10)

TypeError: lagrangian_relaxation() takes exactly 3 arguments (4 given)

In [24]:
# For the 88 Node cluster
iterator(demand = demand_88, distance = distance_88, supply_points = 10)

TypeError: lagrangian_relaxation() takes exactly 3 arguments (4 given)